<center> <h1> UGCA Assignment 3 </h1> </center>
<center> <h3> Matt Viteri, Colin Chu, Darasimi Oluwaniyi, Vikram Gupta, & Yeggi Lee </h3> </center>

<b> High level description: </b> The objective of this group assignment is to create the building blocks of a crowdsourced recommendation system. This recommendation system should accept user inputs about desired attributes of a product and come up with 3 recommendations. In this case, we used beer recommendations from the following website: https://www.beeradvocate.com/beer/top-rated/

In [4]:
import numpy as np
import pandas as pd
import re
import textblob
import nltk
import matplotlib.pyplot as plt
import random

from sklearn import manifold
from configparser import ConfigParser
from IPython.display import display
from collections import Counter
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

stop_words = set(stopwords.words('english'))

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/ylee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### TO KEEP IN MIND: 

Your submission should include the following: 
1. Names of all team members inside the document (only one submission per team)
2. All scripts 
3. The average sentiment and similarity scores for the three products you recommended in task E.
4. Your analyses and answer to task F. Make sure you show the ratings, similarity scores and sentiments for the products you recommend in tasks E and F. 

#### Guidelines
1.    For similarity analysis, use spacy (instead of simple cosine similarity). The script should accept as input a file with the product attributes, and calculate similarity scores (between 0 and 1) between these attributes and each review. That is, the output file should have 3 columns – product_name, product_review and similarity_score. 
2.    The same beer may appear on multiple rows (in the 300 rows with highest similarity scores) since there are multiple reviews for each beer. Reduce the number of rows by averaging the user_rating of each beer (can do it in Excel). After this operation, each beer will be present only in one row. The user_rating column will now have average ratings for each beer. 
3.    For task F, you do the same averaging, but with ALL rows of data (that is, as many rows as there are reviews).   

### Task A
Extract about 6k reviews of beers from Beeradvocate.com. It is a single-page listing of 250 top-rated beers (avoids the pagination feature, which you need in cases where listings go on for many pages). Scrape 25 reviews and well as the ratings provided by users for each beer. The way beeradvocate.com organizes reviews is that it provides 25 reviews per page, which means you can avoid the pagination part for the reviews as well. The CSV output file should have 3 columns: product_name, product_review, and user_rating. 
- The scrapper is in the file, "scraper.py" and the output is in "output.csv"
- Approximately ____k reviews were scraped

In [2]:
df = pd.read_csv('output.csv')
df.head(10)

,beer,look,smell,taste,feel,overall,comment
0,Kentucky Brunch Brand Stout,4.75,4.75,4.75,4.75,5.00,"2016 Silver Wax. Aroma has whiskey, maple, tof..."
1,Kentucky Brunch Brand Stout,4.50,4.75,4.75,4.75,4.75,The beer pours Pitch Black with a frothy tan h...
2,Kentucky Brunch Brand Stout,4.75,4.75,4.50,5.00,4.75,Probably the smoothest beer I have ever had. S...
3,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,"Dark black, very thick, a little bit of tan he..."
4,Kentucky Brunch Brand Stout,4.50,5.00,5.00,5.00,5.00,Poured black as ink with thin ruby edges at 58...
5,Kentucky Brunch Brand Stout,5.00,5.00,4.50,5.00,4.75,she’s got heat....but man is she somethin nice...
6,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,Amazing brew. The maple aroma pours out of thi...
7,Kentucky Brunch Brand Stout,5.00,5.00,3.75,4.50,4.25,Finally got to try the white whale. Pours like...
8,Kentucky Brunch Brand Stout,5.00,5.00,4.75,5.00,4.75,Had this on tap at the 12-15-18 release in Dec...
9,Kentucky Brunch Brand Stout,4.75,5.00,4.75,5.00,5.00,Had this on tap At the KBBS release 12/15/18 p...


### Task B
Assume that a customer, who will be using this recommendation system, has specified 3 attributes in a beer. One website describes multiple attributes of beer: https://www.dummies.com/food-drink/drinks/beer/beer-for-dummies-cheat-sheet/
- <b> Aggressive: </b> Boldly assertive aroma and/or taste
- <b>Balanced:</b> Malt and hops in similar proportions; equal representation of malt sweetness and hop bitterness in the flavor — especially at the finish
- <b>Complex:</b> Multidimensional; many flavors and sensations on the palate
- <b>Crisp: </b>Highly carbonated; effervescent
- <b>Fruity:</b> Flavors reminiscent of various fruits or <b>Hoppy: </b> Herbal, earthy, spicy, or citric aromas and flavors of hops or <b>Malty</b>: Grainy, caramel-like; can be sweet or dry
- <b>Robust:</b> Rich and full-bodied

Choose three attributes from the above list. If you choose another product (like electric guitars), choose a different set – e.g., action, neck, pickup (for guitars). It is a good idea to perform a word frequency analysis of the data you have collected to make sure that the above attributes are actually mentioned.   

NOTE: Please don't rerun; takes a long time

TODO: is strong the same thing as aggresive?

In [14]:
same_map = {
    'aggresive': ['bold', 'assertive'],
    'balanced': ['balance', 'multidimensional'],
    'complex': ['complexing'],
    'crisp': ['crispy', 'carbonated', 'carbonation'],
    'fruity': ['fruit', 'tart', 'apple', 'fig', 'orange', 'apples', 'oranges', 'raspberry', 'raspberries', 
               'cherry', 'grapefruit', 'tropical'],
    'hoppy': ['herbal', 'earthy', 'spicy', 'citric', 'hops', 'hop', 'citrus'],
    'malty': ['malt', 'grainy', 'caramel', 'maple'],
    'robust': ['rich', 'full-bodied']
}

same_inv_map = {word: root for root, words in same_map.items() for word in words}

def splitter(data):
    words = re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", data.lower()).split()
    return [textblob.Word(word).lemmatize() for word in words]

def convert_same(text):
    return [(same_inv_map[word] if word in same_inv_map else word) for word in splitter(text)]

def wordCount(data):
    filtered = [w for w in convert_same(' '.join(data['comment'])) if w not in stop_words]
    return Counter(filtered).most_common(100)

df['split_comments'] = df['comment'].map(convert_same)

wordCount(df)

[('fruity', 130750),
 ('beer', 128501),
 ('hoppy', 94796),
 ('head', 72044),
 ('malty', 69812),
 ('taste', 64404),
 ('flavor', 59767),
 ('wa', 56931),
 ('one', 51947),
 ('dark', 51810),
 ('like', 48094),
 ('crisp', 47147),
 ('chocolate', 46025),
 ('nice', 45110),
 ('smell', 44216),
 ('sweet', 40556),
 ('good', 39395),
 ('aroma', 38741),
 ('bottle', 38607),
 ('coffee', 37789),
 ('well', 37653),
 ('bourbon', 36788),
 ('finish', 35075),
 ('bit', 33494),
 ('note', 33272),
 ('glass', 33053),
 ('light', 32644),
 ('really', 31806),
 ('great', 31475),
 ('alcohol', 31350),
 ('vanilla', 29781),
 ('little', 28839),
 ('pours', 28671),
 ('smooth', 27655),
 ('mouthfeel', 27358),
 ('nose', 27269),
 ('lacing', 26411),
 ('balanced', 25953),
 ('black', 25072),
 ('much', 24557),
 ('poured', 24473),
 ('brown', 24244),
 ('ha', 24124),
 ('get', 24077),
 ('body', 23662),
 ('color', 23590),
 ('thick', 22292),
 ('overall', 22099),
 ('lot', 21990),
 ('stout', 21392),
 ('white', 20839),
 ('medium', 20746),
 ('wo

#### Results
The most common attributes out of the list above are: 
- Fruity
- Hoppy
- Malty 
- Crisp
- Balanced
- Robust
- Complex

### Task C 
Perform a similarity analysis with the 3-attribute set and the reviews. See guidelines below on how to do this. From the output file, choose 300 reviews that have the highest similarity scores with the attribute set you have chosen. 

Run the following line in your terminal AFTER you install spacy
<br />
`python -m spacy download en_core_web_lg`

In [1]:
import spacy

nlp = spacy.load("en_core_web_lg")
STOPWORDS = nlp.Defaults.stop_words

In [2]:
df = pd.read_csv('output.csv')
df.head(10)

,beer,look,smell,taste,feel,overall,comment
0,Kentucky Brunch Brand Stout,4.75,4.75,4.75,4.75,5.00,"2016 Silver Wax. Aroma has whiskey, maple, tof..."
1,Kentucky Brunch Brand Stout,4.50,4.75,4.75,4.75,4.75,The beer pours Pitch Black with a frothy tan h...
2,Kentucky Brunch Brand Stout,4.75,4.75,4.50,5.00,4.75,Probably the smoothest beer I have ever had. S...
3,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,"Dark black, very thick, a little bit of tan he..."
4,Kentucky Brunch Brand Stout,4.50,5.00,5.00,5.00,5.00,Poured black as ink with thin ruby edges at 58...
5,Kentucky Brunch Brand Stout,5.00,5.00,4.50,5.00,4.75,she’s got heat....but man is she somethin nice...
6,Kentucky Brunch Brand Stout,5.00,5.00,5.00,5.00,5.00,Amazing brew. The maple aroma pours out of thi...
7,Kentucky Brunch Brand Stout,5.00,5.00,3.75,4.50,4.25,Finally got to try the white whale. Pours like...
8,Kentucky Brunch Brand Stout,5.00,5.00,4.75,5.00,4.75,Had this on tap at the 12-15-18 release in Dec...
9,Kentucky Brunch Brand Stout,4.75,5.00,4.75,5.00,5.00,Had this on tap At the KBBS release 12/15/18 p...


In [41]:
attributes = ['crisp', 'balanced', 'complex']
attstr = ' '.join(attributes)

In [42]:
# ignore stopwords, punctuation and pronouns
def process_text(text):
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            continue
        result.append(token.lemma_)
    return ' '.join(result)

In [43]:
def calculate_similarity(text):
    base = nlp(attstr)
    compare = nlp(process_text(text))
    return base.similarity(compare)

In [107]:
sims = []

for i, r in df.head(6000).iterrows():
    s = calculate_similarity(r.comment)
    d = {
        'product_name': r.beer,
        'product_review': r.comment,
        'user_rating': r.overall,
        'similarity_score': s
    }
    sims.append(d)

In [108]:
sims_df = pd.DataFrame(sims)
sims_df.sort_values(by=['similarity_score'], ascending=False, inplace=True)
sims_df[0:300]

,product_name,product_review,similarity_score,user_rating
671,Oude Geuze Vintage,This is a pinnacle of gooziness. Extremely com...,0.785470,5.00
5003,Haze,"A citrus bomb, but not overly citrusy. There i...",0.769093,4.50
2209,Sip Of Sunshine,y excellent DIPA. Somewhat pale orange color w...,0.762809,4.50
1434,Darkstar November,My favorite beer. So complex and nicely balanc...,0.762789,5.00
5963,Flora,"Flora is clean, crisp, light, and tart, with s...",0.762249,4.75
672,Nectarine Premiere,"L: Radiant shining hues of apricots, tangerine...",0.758755,4.75
763,Beatification,"Bottle. A: Golden color, totally clear, with ...",0.753764,4.50
4137,Peche 'n Brett,This is exceptional and yet so simple. Fresh p...,0.753475,4.50
3531,Nelson,"Deep yellow color, moderate clarity. Bright wh...",0.753122,4.25
543,Dorothy (Wine Barrel Aged),"Yeasty, fairly dry, and a reasonable barrel pr...",0.752609,4.25


In [109]:
sims_df.to_csv('similarity_score.csv', index=None)

### Task D. 
Perform sentiment analysis on these 300 reviews and sort them (high to low) by the sentiment scores.

In [22]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score['compound']

In [23]:
df_sims_sentiment = pd.read_csv('similarity_score.csv')
df_sims_sentiment.head()
df_sims_sentiment['Sentiment Score'] = df_sentiment['product_review'].map(lambda x: sentiment_analyzer_scores(x))

In [24]:
df_sims_sentiment.sort_values(by=['Sentiment Score'], ascending=False, inplace=True)
print(df_sims_sentiment.head())
df_sims_sentiment.to_csv('similarity_score&sentiment_score.csv', index=None)

                                           product_name  \
2176                                      Beatification   
5400  BBADL (Bourbon Barrel Aged Dark Lord Imperial ...   
40                                       Peche 'n Brett   
389                                            Sure Bet   
1040                                 Oude Geuze Vintage   

                                         product_review  similarity_score  \
2176  Batch pH1  Pours a bright, peachy, warm golden...          0.669302   
5400  I promised not to review anything from Dark Lo...          0.586601   
40    E: Beeswax coated pry-off crown cap. Purchased...          0.736755   
389   Currently #175 in the Top 250 beers list and #...          0.706466   
1040  Presentation: Managed to snaffle two bottles o...          0.688740   

      user_rating  Sentiment Score  
2176         5.00           0.9995  
5400         5.00           0.9994  
40           4.75           0.9994  
389          4.00           0.9993

In [28]:
df_sentiment_300 = pd.read_csv('similarity_score.csv')
df_sentiment_300 = df_sentiment_300[0:300]
print(df_sentiment_300.shape)
df_sentiment_300.head()

(300, 4)


,product_name,product_review,similarity_score,user_rating
0,Oude Geuze Vintage,This is a pinnacle of gooziness. Extremely com...,0.785470,5.00
1,Haze,"A citrus bomb, but not overly citrusy. There i...",0.769093,4.50
2,Sip Of Sunshine,y excellent DIPA. Somewhat pale orange color w...,0.762809,4.50
3,Darkstar November,My favorite beer. So complex and nicely balanc...,0.762789,5.00
4,Flora,"Flora is clean, crisp, light, and tart, with s...",0.762249,4.75


In [30]:
df_sentiment_300['Sentiment Score'] = df_sentiment_300['product_review'].map(lambda x: sentiment_analyzer_scores(x))
df_sentiment_300.sort_values(by=['Sentiment Score'], ascending=False, inplace=True)
df_sentiment_300.head()

,product_name,product_review,similarity_score,user_rating,Sentiment Score
40,Peche 'n Brett,E: Beeswax coated pry-off crown cap. Purchased...,0.736755,4.75,0.9994
160,Society & Solitude #4,"Enjoyed on tap at Armsby Abbey in Worcester, M...",0.718309,5.00,0.9990
99,Sang Noir,750ml brown glass bottle with appealing label ...,0.724068,4.50,0.9990
120,Green,My dad showed up today. He's on a modified car...,0.721891,4.75,0.9990
83,Lou Pepe - Framboise,2008 vintage purchased in Brussels from Bier T...,0.726821,4.50,0.9984


In [31]:
df_sentiment_300.to_csv('taskD.csv', index=None)

### Task E. 

Based on tasks C and D, <b> recommend 3 beers </b> to the customer. Note that in task D, multiple reviews may refer to the same beer. Show the average similarity and sentiment scores for each of the 3 recommended products. 

### Task F. 
How would your recommendations differ if you ignored the similarity and sentiment scores and simply chose the 3 highest rated beers from your entire dataset? To do this, you need to calculate the average rating for each beer mentioned (remember each beer in your data has multiple reviews, and you need to get the average rating for each beer). Would these three beers meet the requirements of the user looking for recommendations? Why or why not? Justify your answer. Use the user_rating data, similarity and sentiment scores to answer this question. 